In [77]:
import numpy as np

In [78]:
class ANN:
    #List of List for each layer along with their information
    #Shape: [ [layerType,activationF(),neuronCount],[layerType,activationF(),neuronCount]]
    layer_info = None
    #List of weight matrices of each layer
    #Shape: dxk (where d is no of input)(k is no of output)
    w_l_ij= None
    #List of bias vector of each layer
    #Shape: 1xk (where k is no of output/neuron)
    b_l_j = None
    #List of output of each neuron
    #Shape: 1xd (where d is dimension)
    a_l_ij = None
    #List of d(a_l+1)/da_l (where a_l+1 is output of layer)(a_l is input of layer)
    #Shape: 1xd (same shape as input of layer)
    delta_l_ij = None
    #List of matrices of dl/dw for each layer
    dl_db = None
    dl_dw = None
    #Dictionary for differentiation function for a input function. diff[f()] => df()
    diff = {}
    input_featureCount = None
    input_count = None
    def __init__(self,xShape,rseed = 10,listLayer= None):
        np.random.seed(rseed)
        inputCount,inputDim = xShape
        self.input_featureCount = inputDim
        self.input_count = inputCount
        self.layer_info = [['None','None',self.input_featureCount]]
        self.w_l_ij = ['None']
        self.b_l_j = ['None']
        self.optimizer = None
        self.diff[ANN.activation_linear] = ANN.diff_activation_linear
        self.diff[ANN.activation_sigmoid] = ANN.diff_activation_sigmoid
        self.diff[ANN.activation_tanh] = ANN.diff_activation_tanh

    def compile(self,optimizer,lossFunction):
        self.optimizer = optimizer
        self.layer_info.append(['Loss',lossFunction])

    def __str__(self):
        return

    def addLayers_Dense(self,neuronCount,activationFunction):
            self.layer_info.append(['Dense',activationFunction,neuronCount])
            w = np.random.rand(self.layer_info[-2][2],neuronCount)
            b = np.random.rand(1,neuronCount)
            self.w_l_ij.append(w)
            self.b_l_j.append(b)

    def addLayers_softmax(self):
        self.layer_info.append(['Softmax',self.layer_info[-1][2]])
        self.w_l_ij.append('None')
        self.b_l_j.append('None')

    def forward_layer_dense(self,layerNo):
        if layerNo < 1:
            print("ERRRROR")
        
        #Output = activationFunction(a_l-1 @ w_l + b_l)
        output = self.layer_info[layerNo][1](self.a_l_ij[layerNo-1] @ self.w_l_ij[layerNo] + self.b_l_j[layerNo]) 
        self.a_l_ij.append(output)
    
    def backward_layer_dense(self,layerNo):
        #at each layer we need to calculate da_l/da_l-1, and da_l/dw_l
        #Shape: dxk
        da_1da = self.diff[self.layer_info[layerNo][1]](self.a_l_ij[layerNo]).T @ self.w_l_ij[layerNo].T
        #Shape: dxk
        da_l1_dw_l = np.zeros(self.w_l_ij[layerNo].shape)
        for i in range(da_l1_dw_l.shape[0]):
            for j in range(da_l1_dw_l.shape[1]):
                da_l1_dw_l[i][j] = self.diff[self.layer_info[layerNo][1]](self.a_l_ij[layerNo][0][j],'SINGLE') * self.a_l_ij[layerNo-1][0][i]

        #multiply both by dL/da_l to get dL/da_l-1 and dL/dw_l
        self.delta_l_ij[layerNo] = self.delta_l_ij[layerNo+1] @ da_1da

        #print("AAA:",da_l1_dw_l,"\n", self.delta_l_ij[layerNo+1])
        temp = np.zeros(self.w_l_ij[layerNo].shape)
        for i in range(temp.shape[0]):
            for j in range(temp.shape[1]):
                temp[i][j] = da_l1_dw_l[i][j] * self.delta_l_ij[layerNo+1][0][j]

        self.dl_dw.append(temp)
        self.dl_db.append(self.delta_l_ij[layerNo+1])
    
    def forward_layer_softmax(self,layerNo):
        if layerNo < 1: 
            print("ERRORRR")
        denom = np.sum( np.exp(self.a_l_ij[layerNo-1]) )
        output = np.exp(self.a_l_ij[layerNo-1]) / denom
        self.a_l_ij.append(output)

    def backward_layer_softmax(self,layerNo):
        pass

    #Create cross entropy loss 
    def forward_layer_loss(self,layerNo,y):
        y_hat  =  self.a_l_ij[-1]
        lossFunction = self.layer_info[layerNo][1]
        return lossFunction(y_hat,y)

    def forwardProp(self,x,y):
        self.a_l_ij = []
        self.a_l_ij.append(x)
        for layerNo in range(len(self.layer_info)):
            if self.layer_info[layerNo][0] == 'Dense':
                self.forward_layer_dense(layerNo)
            if self.layer_info[layerNo][0] == 'Softmax':
                self.forward_layer_softmax(layerNo)
            if self.layer_info[layerNo][0] == 'Loss':
                cost = self.forward_layer_loss(layerNo,y)
                return cost
            
    #Handle softmax backpropogation
    def backwardProp(self,y):
        self.dl_dw = []
        self.dl_db = []
        lastIndex = len(self.layer_info) - 2
        self.delta_l_ij = [i for i in range(0,lastIndex+1)]
        
        #MSE LOSS And Dense
        self.delta_l_ij[lastIndex] = 2 * ( (self.a_l_ij[lastIndex] - y) @ self.w_l_ij[lastIndex].T) 
        self.dl_dw.append(2*(self.a_l_ij[lastIndex-1].T @ self.a_l_ij[lastIndex] - y))
        self.dl_db.append(2*(self.a_l_ij[lastIndex] - y))

        #BackPropogation
        for layerNo in range(lastIndex-1,0,-1):
            if self.layer_info[layerNo][0] == 'Dense':
                self.backward_layer_dense(layerNo)
            if self.layer_info[layerNo][0] == 'Softmax':
                self.backward_layer_softmax(layerNo)
                
        self.dl_dw.append(['None'])
        self.dl_db.append(['None'])
        self.dl_dw.reverse()
        self.dl_db.reverse()


            
    def fit(self,x,y):
        #Hyperparameters if required: 
        epoch = 100
        eta = 4e-5
        self.optimizer(self,x,y,eta,epoch)

    @staticmethod    
    def optimizer_gradientDescent(Obj,x,y,eta,epoch):
        for i in  range(epoch):
            errorSum = 0
            print("Epoch:",i,end="\t")
            for j in range(1):
                #x and y both are 2d matrix
                x_1xd = x[j:j+1]
                y_1xk = y[j:j+1]
                errorSum += Obj.forwardProp(x_1xd,y_1xk)
                Obj.backwardProp(y_1xk)
                for index in range(1,len(Obj.w_l_ij)):
                    Obj.w_l_ij[index] = Obj.w_l_ij[index] - (eta * Obj.dl_dw[index])
                    Obj.b_l_j[index] = Obj.b_l_j[index] - (eta * Obj.dl_db[index] )
            print("Error:",errorSum)

    @staticmethod
    def loss_MSE(P,y):
        return np.sum((P - y)**2)   
      
    @staticmethod
    def activation_linear(z):
        return z
    
    @staticmethod
    def diff_activation_linear(x,flg='None'):
        if(flg != 'SINGLE'):
            return np.identity(x.shape[1])
        else:
            return x
    
    @staticmethod
    def activation_sigmoid(z):    
        return 1 / (1 + np.exp(-z))
    
    @staticmethod    
    def diff_activation_sigmoid(a,flg='None'):
        if(flg !='SINGLE'):
            return np.diag(a * (1-a))
        else:
            return a * (1- a)

    @staticmethod
    def activation_tanh(z):
        sinh = np.exp(z) - np.exp(-z)
        cosh = np.exp(z) + np.exp(-z)
        return sinh / cosh
    
    @staticmethod
    def diff_activation_tanh(z,flg='None'):
        if(flg != 'SINGLE'):
            return np.diag(1 - (z*z))
        else:
            return 1 - (z * z)

    @staticmethod
    def activation_relu(z):
        pass

In [79]:
import pandas as pd
import numpy as np
import sklearn.datasets as sk

In [80]:
dictt = sk.fetch_california_housing()
x = dictt.data
y = np.array(dictt.target)
y = y.T
x

array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]])

In [81]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer().fit(x)
x = transformer.transform(x)
x

array([[ 0.0238481 ,  0.1174473 ,  0.02000651, ...,  0.00732056,
         0.10850985, -0.3501362 ],
       [ 0.00345241,  0.00873354,  0.00259434, ...,  0.00087745,
         0.01574533, -0.05082923],
       [ 0.01409202,  0.10097076,  0.01609345, ...,  0.00544128,
         0.07349506, -0.23735895],
       ...,
       [ 0.00167455,  0.01674553,  0.00512762, ...,  0.00229082,
         0.03883978, -0.11940547],
       [ 0.00248251,  0.02393168,  0.00708579, ...,  0.00282289,
         0.05242368, -0.16129955],
       [ 0.00171478,  0.0114864 ,  0.00377236, ...,  0.00187873,
         0.02826371, -0.08703817]])

In [82]:
model = ANN(x.shape,20)

In [83]:
model.addLayers_Dense(9,ANN.activation_linear)
model.addLayers_Dense(4,ANN.activation_linear)
model.addLayers_Dense(1,ANN.activation_linear)

In [84]:
model.compile(ANN.optimizer_gradientDescent,ANN.loss_MSE)

In [85]:
model.layer_info

[['None', 'None', 8],
 ['Dense', <function __main__.ANN.activation_linear(z)>, 9],
 ['Dense', <function __main__.ANN.activation_linear(z)>, 4],
 ['Dense', <function __main__.ANN.activation_linear(z)>, 1],
 ['Loss', <function __main__.ANN.loss_MSE(P, y)>]]

In [86]:
import copy
w_old = copy.deepcopy(model.w_l_ij)
b_old = copy.deepcopy(model.b_l_j)
dldw = copy.deepcopy(model.dl_dw)

In [87]:
model.fit(x,y)

Epoch: 0	Error: 9.974453712895581
Epoch: 1	Error: 9.65004927683404
Epoch: 2	Error: 9.33541947232648
Epoch: 3	Error: 9.030237327332655
Epoch: 4	Error: 8.734189581401962
Epoch: 5	Error: 8.44697598843659
Epoch: 6	Error: 8.168308660987657
Epoch: 7	Error: 7.897911453251741
Epoch: 8	Error: 7.635519380152611
Epoch: 9	Error: 7.380878070091701
Epoch: 10	Error: 7.133743249133179
Epoch: 11	Error: 6.893880254556057
Epoch: 12	Error: 6.661063575859092
Epoch: 13	Error: 6.435076421444371
Epoch: 14	Error: 6.215710309334814
Epoch: 15	Error: 6.002764680399225
Epoch: 16	Error: 5.796046532667597
Epoch: 17	Error: 5.59537007542005
Epoch: 18	Error: 5.400556401825047
Epoch: 19	Error: 5.211433178988075
Epoch: 20	Error: 5.027834354350561
Epoch: 21	Error: 4.849599877451541
Epoch: 22	Error: 4.676575436131762
Epoch: 23	Error: 4.50861220632183
Epoch: 24	Error: 4.345566614613669
Epoch: 25	Error: 4.187300112867521
Epoch: 26	Error: 4.033678964156047
Epoch: 27	Error: 3.884574039392673
Epoch: 28	Error: 3.7398606240338177

In [88]:
model.a_l_ij

[array([[ 0.0238481 ,  0.1174473 ,  0.02000651,  0.00293277,  0.92239103,
          0.00732056,  0.10850985, -0.3501362 ]]),
 array([[1.10431501, 1.19502222, 0.61940572, 1.04338856, 1.14509088,
         1.1403652 , 0.61092989, 0.78603184, 0.14658372]]),
 array([[3.52320829, 5.01334637, 3.54803971, 4.53163987]]),
 array([[4.55022833]])]

In [89]:
w_old

['None',
 array([[0.5881308 , 0.89771373, 0.89153073, 0.81583748, 0.03588959,
         0.69175758, 0.37868094, 0.51851095, 0.65795147],
        [0.19385022, 0.2723164 , 0.71860593, 0.78300361, 0.85032764,
         0.77524489, 0.03666431, 0.11669374, 0.7512807 ],
        [0.23921822, 0.25480601, 0.85762553, 0.94977903, 0.56168686,
         0.17878052, 0.77025193, 0.49238104, 0.63125307],
        [0.83949792, 0.4610394 , 0.49794007, 0.67941112, 0.65078591,
         0.26879524, 0.06732467, 0.77144514, 0.48098413],
        [0.32920641, 0.51064106, 0.26362883, 0.31051155, 0.62685344,
         0.55744981, 0.31857956, 0.39484322, 0.25797459],
        [0.58224112, 0.16162871, 0.59813382, 0.82582358, 0.15639172,
         0.73430052, 0.40864343, 0.7786879 , 0.80397057],
        [0.78607144, 0.59228702, 0.6644892 , 0.64656729, 0.42563648,
         0.51356833, 0.50125784, 0.03708381, 0.7081161 ],
        [0.6204306 , 0.77780853, 0.45940947, 0.37980555, 0.2918922 ,
         0.55722886, 0.0841636 , 

In [90]:
model.b_l_j

['None',
 array([[0.88794282, 0.87515968, 0.33956575, 0.69250839, 0.51831089,
         0.65416821, 0.26329065, 0.60062043, 0.03991556]]),
 array([[0.35539385, 0.08285786, 0.72672197, 0.98908231]]),
 array([[0.9617482]])]

In [91]:
b_old

['None',
 array([[0.89123753, 0.87858258, 0.34475163, 0.70310053, 0.5273376 ,
         0.65931696, 0.26829063, 0.60776708, 0.04815737]]),
 array([[0.35499545, 0.0846383 , 0.73116878, 0.99660085]]),
 array([[0.97250331]])]

In [92]:
model.dl_dw

[['None'],
 array([[ 1.88820966e-04,  1.92778871e-04,  2.11484079e-04,
          8.45697560e-04,  7.29438774e-04,  4.07467014e-04,
          1.85615267e-04,  3.98153768e-04,  8.76477254e-05],
        [ 9.29906743e-04,  9.49398657e-04,  1.04151819e-03,
          4.16489693e-03,  3.59234490e-03,  2.00669624e-03,
          9.14119295e-04,  1.96083031e-03,  4.31648097e-04],
        [ 1.58404556e-04,  1.61724897e-04,  1.77416958e-04,
          7.09467538e-04,  6.11936413e-04,  3.41829790e-04,
          1.55715250e-04,  3.34016777e-04,  7.35289055e-05],
        [ 2.32206678e-05,  2.37073997e-05,  2.60077132e-05,
          1.04001491e-04,  8.97043151e-05,  5.01091396e-05,
          2.28264400e-05,  4.89638230e-05,  1.07786691e-05],
        [ 7.30317003e-03,  7.45625286e-03,  8.17972823e-03,
          3.27096783e-02,  2.82130502e-02,  1.57599071e-02,
          7.17918081e-03,  1.53996917e-02,  3.39001676e-03],
        [ 5.79616669e-05,  5.91766100e-05,  6.49184780e-05,
          2.59600622e-04

In [93]:
dldw

In [94]:
np.zeros((1,2))

array([[0., 0.]])

In [95]:
x = np.array([1,2,3,4])
np.diag(x)

array([[1, 0, 0, 0],
       [0, 2, 0, 0],
       [0, 0, 3, 0],
       [0, 0, 0, 4]])

MSE -> Generalise for any loss
Softmax backProp
Stopping condition stochastic gd
